**Import packages and log in**

In [ ]:
# !pip install celonis_ml
import csv
import os
import numpy as np
import pandas as pd
import copy
import sys
import yaml
import re
from collections import OrderedDict
from pycelonis import get_celonis, pql
from pycelonis.pql import PQL, PQLColumn
from pycelonis.utils import parquet_utils as pu

login = {
        "celonis_url": "",
        "api_token": "",
    }
celonis = get_celonis(**login)

In [ ]:
package = celonis.packages.find('31af4c4d-4ddd-40ae-97a6-9d1146345e6f')
source_analysis = package.analyses.find('0c191ff3-5ef8-47c9-92dd-f5170e342f2a')

In [ ]:
class quoted(str):
    pass

def quoted_presenter(dumper, data):
    return dumper.represent_scalar('tag:yaml.org,2002:str', data, style='"')
    
yaml.add_representer(quoted, quoted_presenter)

class literal(str):
    pass

def literal_presenter(dumper, data):
    return dumper.represent_scalar('tag:yaml.org,2002:str', data, style='>')
    
yaml.add_representer(literal, literal_presenter)

def ordered_dict_presenter(dumper, data):
    return dumper.represent_dict(data.items())

yaml.add_representer(OrderedDict, ordered_dict_presenter)


def add_parameters(data, km_kpi):
    if len(data['parameters']) != 0:
        km_parameters = []
        for parameter, i in zip(data['parameters'], range(len(data['parameters']))):
            km_parameter = {
                'id': 'p'+str(i+1),
                'displayName': parameter['name'],
                'defaultValue': 0
            }
            km_parameters.append(km_parameter)
        km_kpi['parameters'] = km_parameters
    return km_kpi

def clean_formula_names(saved_formulas):
    to_return = []
    for data in saved_formulas:
        data['clean_name'] = data['name'].replace(')', '').replace('(', '')
        data['clean_name'] = re.sub("[^0-9a-zA-Z]+", "_", data['clean_name'])
        to_return.append(data)
    return to_return

def clean_template(clean_formulas):
    to_return = []
    for data in clean_formulas:
        tmp_template = data['template']
        for tmp_data in clean_formulas:
            name = tmp_data['name']
            clean_name = tmp_data['clean_name']
            tmp_template = tmp_template.replace(name, clean_name)
        data['clean_template'] = tmp_template
        to_return.append(data)
    return clean_formulas
            
def clean_variables(var_list):
    to_return = []
    for y in var_list:
        tmp_var_query = y['value']
        for x in var_list:            
            full_var_name0, full_var_name1, full_var_name2, full_var_name3 = '<%='+x['name']+'%>', '<%= '+x['name']+' %>', '<%= '+x['name']+'%>', '<%='+x['name']+' %>'
            tmp_var_query = tmp_var_query.replace(full_var_name0, '${'+x['name']+'}').replace(full_var_name1, '${'+x['name']+'}').replace(full_var_name2, '${'+x['name']+'}').replace(full_var_name3, '${'+x['name']+'}')
        var_data = {'id': y['name'],
                    'displayName': y['name'].replace('_', ' '),
                    'description':"",
                    'value': tmp_var_query
        }
        to_return.append(var_data)
    return to_return

def replace_variables(clean_formulas, var_list):
    to_return = []
    for data in clean_formulas:
        tmp_data = copy.copy(data['clean_template'])
        for x in var_list:
            full_var_name0, full_var_name1, full_var_name2, full_var_name3 = '<%='+x['name']+'%>', '<%= '+x['name']+' %>', '<%= '+x['name']+'%>', '<%='+x['name']+' %>'
            tmp_data = tmp_data.replace(full_var_name0, '${'+x['name']+'}').replace(full_var_name1, '${'+x['name']+'}').replace(full_var_name2, '${'+x['name']+'}').replace(full_var_name3, '${'+x['name']+'}')
        data['clean_template'] = tmp_data
        to_return.append(data)
    return to_return
        
def saved_formulas_to_yaml(source_analysis):
    """Given a Celonis Analysis object, saves a 'data.yml' file """
    
    saved_formulas = copy.copy(source_analysis.saved_formulas)
    var_list = copy.copy(source_analysis.analysis.draft.variables)
    km_kpis = []
    list_of_data = [formula.data for formula in saved_formulas]    
        
    clean_data = clean_formula_names(list_of_data)
    clean_data = clean_template(clean_data)
    clean_data = replace_variables(clean_data, var_list)
    for data in clean_data:
        description = data['description'].replace('\'', '').replace('"', '')
        pql = '\n'+data['clean_template'].replace('.id', '."id"').replace('.Id', '."Id"').replace('.ID', '."ID"')
        km_kpi =  OrderedDict(id=data['clean_name'],
                displayName=data['name'].replace('_', ' '),
                description=quoted(description),
                pql=literal(pql),
                )
        km_kpi = add_parameters(data, km_kpi)
        km_kpis.append(km_kpi)
        
    km_kpis = {'kpis':km_kpis}
    
    with open('kpis.yml', 'w') as outfile:
        yaml.dump(km_kpis, outfile, sort_keys=False)
    return

def variables_to_yaml(source_analysis):
    var_list = copy.copy(source_analysis.analysis.draft.variables)
    var_list = clean_variables(var_list)
    km_vars = []
    for var in var_list:
        km_var = OrderedDict(id=var['id'],
                             displayName=var['displayName'],
                             description=var['description'],
                             value=literal(var['value'].replace('"', '') )
        )
        km_vars.append(km_var)
    km_vars = {'variables':km_vars}
    with open('variables.yml', 'w') as outfile:
        yaml.dump(km_vars, outfile, sort_keys=False)
    return

In [ ]:
saved_formulas_to_yaml(source_analysis)

In [ ]:
variables_to_yaml(source_analysis)